### **Introducción a LangChain**

LangChain es una librería de alto nivel diseñada para facilitar la construcción de aplicaciones impulsadas por modelos de lenguaje (LLM), proporcionando abstracciones sobre prompts, cadenas (Chains), agentes y herramientas. 

Su objetivo es ofrecer componentes reutilizables que orquesten flujos de llamada a LLM, gestión de contexto, almacenamiento de estados conversacionales y encadenamiento de tareas complejas sin tener que lidiar con detalles de bajo nivel de la API.

In [ ]:
!pip install transformers langchain accelerate

#### **Agentes LangChain  y herramientas**

Los **Agents** en LangChain combinan LLMs con "herramientas" externas (por ejemplo, buscadores Web, bases de datos, funciones personalizadas). 

Funcionan así: el agente decide, a partir de un prompt y su memoria, qué herramienta invocar y con qué argumentos; tras ejecutar la herramienta, incorpora la respuesta al contexto y continúa el flujo. 

Esto permite construir asistentes que responden de manera dinámica, consultan APIs o ejecutan código.

El patrón general para una aplicación con LangChain incluye:

1. **PromptTemplate**: definiciones parametrizadas de prompts.
2. **LLMChain**: encadena el prompt con la llamada al modelo y procesa la respuesta.
3. **Memory** (opcional): guarda fragmentos de la conversación o metadatos.
4. **Agents & Tools** (opcional): permite reactividad a eventos o consultas externas.



**Ejemplo 1: Generación de una historia interactiva**

Este ejemplo genera una historia interactiva donde el lector puede elegir diferentes opciones y el modelo continuará la historia según la elección del lector.

In [ ]:
from transformers import pipeline
import torch

# 1. Detectae el dispositivo y tipo de dato óptimo
if torch.cuda.is_available():
    dispositivo = 0                     # usa GPU 0
    dtype = torch.bfloat16             # bfloat16 en GPU para velocidad y menor uso de memoria
else:
    dispositivo = -1                   # usa CPU
    dtype = torch.float32              # float32 en CPU para evitar conversiones lentas

# 2. Crea el pipeline de generación sólo una vez
generate_text = pipeline(
    "text-generation",                 
    model="aisquared/dlite-v1-355m",   
    torch_dtype=dtype,                 
    trust_remote_code=True,            
    device=dispositivo,                
)

# 3. Warm-up: primera llamada rápida para cache interno
_ = generate_text("Hola")

# 4. Función optimizada para continuar la historia
def continue_story(prompt: str, choice: str) -> str:
    """
    Genera la siguiente parte de la historia combinando el prompt y la elección del usuario.
    - max_new_tokens: límite bajo para tokens nuevos (reduce tiempo de generación)
    - do_sample=False: decodificación greedy (más rápida y determinista)
    """
    salida = generate_text(
        f"{prompt} {choice}",
        max_new_tokens=60,
        do_sample=False
    )
    # Algunos pipelines devuelven lista de strings
    if isinstance(salida, list) and isinstance(salida[0], str):
        return salida[0]
    # Otros devuelven lista de dicts con key "generated_text" o "generated_sequence"
    if isinstance(salida, list) and isinstance(salida[0], dict):
        return salida[0].get("generated_text") or salida[0].get("generated_sequence", "")
    # Fallback: cast a str
    return str(salida)

# 5. Flujo interactivo con comentarios en español
story_prompt = (
    "Te encuentras en un bosque oscuro. Delante de ti hay dos caminos. "
    "¿Tomas el camino izquierdo o el camino derecho?"
)
print(story_prompt)

choice1 = input("Ingresa tu elección (izquierdo/derecho): ").strip().lower()
story1 = continue_story(story_prompt, choice1)
print(story1)

story_prompt2 = (
    story1 +
    "\nTe topas con una figura misteriosa. ¿Te acercas a la figura o te escondes detrás de un árbol?"
)
choice2 = input("Ingresa tu elección (acercarse/esconderse): ").strip().lower()
story2 = continue_story(story1, choice2)
print(story2)

story_prompt3 = (
    story2 +
    "\nLa figura te ofrece un objeto mágico. ¿Lo aceptas o lo rechazas?"
)
choice3 = input("Ingresa tu elección (aceptar/rechazar): ").strip().lower()
story3 = continue_story(story2, choice3)
print(story3)

story_prompt4 = (
    story3 +
    "\nCon el objeto mágico en tu mano, sientes un poder. ¿Lo usas ahora o lo guardas para más tarde?"
)
choice4 = input("Ingresa tu elección (usar/guardar): ").strip().lower()
story4 = continue_story(story3, choice4)
print(story4)

story_prompt5 = (
    story4 +
    "\nTe encuentras con un dragón feroz. ¿Luchas contra él o intentas comunicarte?"
)
choice5 = input("Ingresa tu elección (luchar/comunicar): ").strip().lower()
story5 = continue_story(story4, choice5)
print(story5)

print("\nLa historia termina aquí. ¡Gracias por jugar!")


**Ejemplo 2: Generación de un documento técnico**

Este ejemplo utiliza el modelo para generar un documento técnico sobre inteligencia artificial, dividiendo el contenido en secciones específicas.

In [ ]:
from transformers import pipeline
import torch

# 1. Detecta el dispositivo y tipo de dato óptimo
if torch.cuda.is_available():
    dispositivo = 0                     # usa GPU 0
    dtype = torch.bfloat16             # bfloat16 en GPU para velocidad y menor uso de memoria
else:
    dispositivo = -1                   # usa CPU
    dtype = torch.float32              # float32 en CPU para evitar conversiones lentas

# 2. Crea el pipeline de generación de texto una sola vez
generate_text = pipeline(
    "text-generation",
    model="aisquared/dlite-v1-355m",
    torch_dtype=dtype,
    trust_remote_code=True,
    device=dispositivo,
)

# 3. Warm-up: llama brevemente para inicializar caches internas
_ = generate_text("Hola")

def _extract_text(output) -> str:
    """
    Función interna para extraer el texto generado
    Soporta lista de strings, lista de dicts o un string directo.
    """
    if isinstance(output, list):
        first = output[0]
        if isinstance(first, str):
            return first
        if isinstance(first, dict):
            return first.get("generated_text") or first.get("generated_sequence", "")
    if isinstance(output, dict):
        return output.get("generated_text") or output.get("generated_sequence", "")
    return str(output)

# 4. Función optimizada para generar secciones del documento
def generate_section(title: str) -> str:
    """
    Genera una sección detallada sobre el título dado,
    usando un prompt contextualizado en IA.
    """
    prompt = f"Escriba una sección detallada sobre {title} en el contexto de inteligencia artificial."
    out = generate_text(
        prompt,
        max_new_tokens=100,   # límite de tokens a generar por sección
        do_sample=False        # decodificación greedy (más rápida y determinista)
    )
    return _extract_text(out)

# 5. Generación del documento con comentarios en español
title = "Una guía completa sobre la inteligencia artificial"
print(title)
print("=" * len(title))

for section_title in [
    "Introduction",
    "History of Artificial Intelligence",
    "Machine Learning",
    "Deep Learning",
    "Applications of AI",
    "Future of AI"
]:
    print(f"\n{section_title}")
    print("-" * len(section_title))
    texto = generate_section(section_title)
    print(texto)

# 6. Sección de conclusión
print("\nConclusion")
print("-" * len("Conclusion"))
out_conclusion = generate_text(
    "Escriba una conclusión para una guía completa sobre inteligencia artificial.",
    max_new_tokens=80,
    do_sample=False
)
print(_extract_text(out_conclusion))


**Ejemplo 3: Explicaciones y ejemplos de funciones matemáticas**

Este ejemplo proporcionará una explicación de varios conceptos matemáticos y ejemplos prácticos para cada uno.

In [ ]:
from transformers import pipeline
import torch

# 1. Detecta dispositivo y tipo de dato óptimo
if torch.cuda.is_available():
    dispositivo = 0                   # usa GPU 0
    dtype = torch.bfloat16           # bfloat16 en GPU para velocidad y menor uso de memoria
else:
    dispositivo = -1                 # usa CPU
    dtype = torch.float32            # float32 en CPU para evitar conversiones lentas

# 2. Crea el pipeline de generación de texto una sola vez
generate_text = pipeline(
    "text-generation",
    model="aisquared/dlite-v1-355m",
    torch_dtype=dtype,
    trust_remote_code=True,
    device=dispositivo,
)

# 3. Warm-up: inicializar caches internas
_ = generate_text("Hola")

def _extract_text(output) -> str:
    """
    Extrae el texto generado de distintos formatos de salida:
    - lista de strings
    - lista de dicts con 'generated_text' o 'generated_sequence'
    - dict único
    - string directo
    """
    if isinstance(output, list):
        first = output[0]
        if isinstance(first, str):
            return first
        if isinstance(first, dict):
            return first.get("generated_text") or first.get("generated_sequence", "")
    if isinstance(output, dict):
        return output.get("generated_text") or output.get("generated_sequence", "")
    return str(output)

# 4. Función optimizada para explicar conceptos matemáticos
def explain_math_concept(concept: str) -> str:
    """
    Genera una explicación detallada del concepto matemático con ejemplos.
    """
    prompt = f"Explica el {concept} con ejemplos."
    out = generate_text(
        prompt,
        max_new_tokens=100,    # límite de tokens para la explicación
        do_sample=False        # decodificación greedy (más rápida)
    )
    return _extract_text(out)

# 5. Función optimizada para generar ejemplos prácticos
def generate_math_examples(concept: str) -> str:
    """
    Proporciona ejemplos prácticos y soluciones paso a paso para el concepto dado.
    """
    prompt = f"Proporciona ejemplo y soluciones paso a paso para el {concept}."
    out = generate_text(
        prompt,
        max_new_tokens=150,    # límite de tokens para los ejemplos
        do_sample=False
    )
    return _extract_text(out)

# 6. Lista de conceptos y generación
math_concepts = ["derivatives", "integrals", "linear algebra", "probability", "statistics"]

for concept in math_concepts:
    print(f"\nConcepto: {concept.capitalize()}")
    print("=" * (len(concept) + 10))

    # Explicación del concepto
    explanation = explain_math_concept(concept)
    print("\nExplicación:")
    print(explanation)

    # Ejemplos prácticos
    examples = generate_math_examples(concept)
    print("\nEjemplos prácticos:")
    print(examples)

    print("\n" + "=" * 80)

# 7. Ejemplo adicional: ecuación cuadrática
quadratic_prompt = (
    "Resuelve la ecuación cuadrática 3x^2 - 4x - 5 = 0."
    "Proporciona una solución y una explicación paso a paso."
)
quadratic_out = generate_text(
    quadratic_prompt,
    max_new_tokens=120,
    do_sample=False
)
print("\nEjemplo de ecuación cuadrática:")
print("-" * 30)
print(_extract_text(quadratic_out))


### **Prompting con LangChain**

**1. Prompt Templates: separación de redacción y variables**

Las **Prompt Templates** son plantillas parametrizables que distinguen claramente la **estructura fija** de un prompt (instrucción, formatos, estilo) del **contenido dinámico** (variables concretas).

* **Desacoplamiento**
  El texto base del prompt queda centralizado en una plantilla; las variables se inyectan al ejecutarlo. De este modo no mezclamos la lógica de generación con valores específicos, lo que facilita su lectura y modificación.

* **Mantenibilidad**
  Al cambiar el tono, corregir errores tipográficos o ajustar el estilo, basta con actualizar la plantilla; todas las invocaciones posteriores heredarán la mejora de forma automática.

* **Reutilización**
  Una misma plantilla puede servir para diferenciar tareas (resúmenes, explicaciones, preguntas) simplemente variando las variables de entrada. Esto reduce la duplicación de prompts en el código y promueve un diseño más limpio.


**2. Few-Shot & ExampleSelectors: ejemplos manuales y automáticos**

Los **Few-Shot Prompts** incluyen ejemplos concretos de pares *(entrada -> salida)* dentro del propio prompt, enseñando al modelo "en vuelo" el patrón deseado. Sin embargo, elegir manualmente qué ejemplos incluir puede resultar laborioso y no siempre óptimo.

* **ExampleSelectors**
  Automatizan la selección de los ejemplos más relevantes desde una librería amplia, basándose en:

  * **Similitud semántica** (mide la cercanía con embeddings).
  * **Coincidencia de palabras clave** o metadatos.

* **Beneficios**

  1. **Adaptabilidad**: el modelo recibe ejemplos alineados con la consulta actual, mejorando su entendimiento.
  2. **Escalabilidad**: permite gestionar grandes bancos de ejemplos sin saturar el contexto.
  3. **Calidad constante**: evita sesgos o errores de selección manual, pues el sistema elige según métricas objetivas.

**3. Output Parsers: formatos rígidos y fiabilidad**

Los **Output Parsers** convierten la respuesta "libre" de un LLM en estructuras de datos controladas (JSON, diccionarios tipados, objetos de validación).

* **Validación temprana**
  Si la salida no encaja en el esquema esperado (por ejemplo, falta un campo obligatorio o el tipo de dato es incorrecto), el parser detecta la anomalía y permite:

  * Solicitar al modelo un reintento con instrucciones aclaratorias.
  * Aplicar reglas de recuperación o fallback.

* **Reducción de errores downstream**
  Al garantizar que el formato de salida sea siempre uniforme, evitamos excepciones e inconsistencias en etapas posteriores (almacenamiento, visualización, integración con otras APIs).

* **Trazabilidad y observabilidad**
  Registrar cuántas veces fallan los parsers o qué validaciones son las más conflictivas ayuda a iterar y optimizar tanto los prompts como los esquemas de datos.


**4. Aplicación en un resumidor de noticias**

Al incorporar estas tres piezas en un **News Articles Summarizer**, elevamos la calidad y consistencia de los resúmenes:

1. **Prompt Templates**

   * Definen el estilo editorial (longitud, tono, nivel de detalle) en un único lugar.
   * Permiten ajustar rápidamente el "brief" que se envía al LLM sin tocar la lógica de orquestación.

2. **Few-Shot & ExampleSelectors**

   * Recurre a resúmenes humanos previos de artículos similares para guiar el estilo y evitar alucinaciones.
   * Selecciona los ejemplos más alineados semánticamente con el tema del artículo, garantizando coherencia temática.

3. **Output Parsers**

   * Fuerzan un formato JSON fijo:

     ```json
     {
       "title": "...",
       "summary": "...",
       "keywords": ["...", "..."],
       "author": "...",
       "date": "YYYY-MM-DD"
     }
     ```
   * Facilitan la integración con sistemas de publicación o bases de datos, minimizando la limpieza de datos manual.


**5. Extensión a Knowledge Graphs**

Para convertir los resúmenes en un **recurso navegable y semántico**, podemos construir grafos de conocimiento:

1. **Extracción de entidades y relaciones**

   * Mediante prompts especializados, pedimos al LLM que identifique nombres de personas, organizaciones, conceptos clave y sus interacciones.

2. **Construcción del grafo**

   * **Nodos**: las entidades detectadas.
   * **Aristas**: relaciones etiquetadas (por ejemplo, "colaboró con", "es parte de", "causó").

3. **Consultas semánticas**

   * Combinamos un *retriever* para localizar fragmentos relevantes en los resúmenes con prompts que interpretan rutas en el grafo:

     > "¿Cómo se relaciona el concepto X con la organización Y?"

4. **Beneficios**

   * Descubrimos conexiones ocultas entre temas.
   * Ofrecemos a los usuarios exploración interactiva: navegar de nodo en nodo, profundizar en relaciones, generar nuevos insights.



### **Ingeniería de prompts**

Además de los conceptos básicos (plantillas, few-shot, parsers), en la práctica es útil trabajar con una especie de "checklist" de ingeniería de *prompts*. Esta sección resume patrones que usaremos tanto con Transformers "puros" como con LangChain y, más adelante, en RAG.  
En las **celdas de código siguientes** verás ejemplos prácticos de:

- Estructurar prompts con secciones y delimitadores.
- Usar few-shot prompting.
- Generar salidas estructuradas (JSON).
- Hacer prompting "grounded" con contexto al estilo RAG y reducir alucinaciones.

#### **1. Formular tareas claramente y estructurar *prompts* con secciones**

Un *prompt* efectivo suele tener partes bien separadas:

- **Contexto**: de qué va la tarea.
- **Instrucción**: qué debe hacer el modelo.
- **Ejemplos (opcional)**.
- **Formato de salida**.

Ejemplo de estructura verbal:

> **Rol:** Eres un asistente que explica temas de NLP a estudiantes de pregrado.  
> **Tarea:** Explica el concepto de *transformer* en lenguaje sencillo.  
> **Restricciones:**  
> - Máximo 3 párrafos.  
> - Incluye un ejemplo concreto.  

En los ejemplos de código veremos cómo convertir estas ideas en prompts reutilizables.


#### **2. Usar *few-shot prompting*/*in-context learning* con buenos ejemplos**

En vez de dar solo una instrucción, añadimos ejemplos bien elegidos:

- Muestran el **estilo** deseado.
- Fijan el **formato** de entrada y salida.
- Guían al modelo cuando la tarea es ambigua.

Esquema típico:

> **Ejemplo 1**  
> Entrada: "Explica *overfitting* a un niño de 10 años."  
> Salida: "Imagina que..."  
>
> **Ejemplo 2**  
> Entrada: "Explica *regularización L2* a un estudiante de CS."  
> Salida: "En modelos lineales..."  
>
> **Ahora tú:**  
> Entrada: "Explica *embeddings* a un estudiante de ingeniería."  
> Salida: ...

En el primer ejemplo de código combinaremos **few-shot** con salidas en formato **JSON**.

#### **3. Aplicar *chain-of-thought* y razonamiento paso a paso**

Cuando la tarea requiere razonamiento (matemático, lógico, planificación), podemos pedir explícitamente:

- "Razona paso a paso antes de dar la respuesta final."
- "Primero explica tu razonamiento y luego da solo la respuesta final."

Ejemplo de instrucción:

> "Resuelve el problema paso a paso. No saltes pasos.  
> Al final, resume la respuesta en una sola línea que empiece con:  
> **Respuesta final:** ..."

Esto ayuda al modelo a "desplegar" su razonamiento en lugar de intentar adivinar directamente la respuesta final.

#### **4. Diseñar *prompts* de auto-verificación y mejora iterativa**

Podemos pedir que el modelo:

1. Produzca un primer borrador.
2. Revise su propia respuesta.
3. Corrija o mejore.

Patrón típico:

1. "Primero, genera una respuesta inicial."  
2. "Luego, revisa tu respuesta buscando errores, omisiones o incoherencias."  
3. "Por último, entrega una versión corregida y mejorada."  

Este patrón es útil para tareas complejas (código, redacción técnica, planes, etc.).

#### **5. Controlar estilo, nivel y rol del modelo según la audiencia**

Definimos explícitamente:

- **Rol:** "Eres un profesor universitario...", "Eres un revisor crítico...", "Eres un asistente pedagógico..."
- **Nivel:** "Dirígete a estudiantes de pregrado...", "Nivel técnico básico...", "Nivel avanzado..."
- **Estilo:** "Usa ejemplos concretos", "Evita ecuaciones", "Sé conciso (máx. 200 palabras)".

Esto permite adaptar el mismo conocimiento a distintas audiencias sin cambiar el modelo ni el código.

#### **6. Generar salidas estructuradas (JSON, tablas) de forma fiable**

Cuando queremos que la salida se procese con código, necesitamos **formato rígido**:

- Especificar el **esquema** (campos, tipos).
- Pedir explícitamente: "No agregues texto fuera del JSON".

Ejemplo de instrucción:

> "Devuelve la respuesta **únicamente** en formato JSON con la forma:  
> `{ "tema": string, "nivel": string, "explicacion": string, "ejemplo": string }`  
> No incluyas texto adicional antes o después del JSON."


#### **7. *Prompting* con contexto (RAG) y reducción de alucinaciones**

En RAG, el modelo debe basarse **solo en los documentos recuperados**. Un patrón típico:

1. Pegamos el contexto (chunks recuperados) entre delimitadores claros.
2. Damos instrucciones explícitas anti-alucinación.

Ejemplo de instrucciones:

- "Usa **exclusivamente** la información del contexto proporcionado."  
- "Si la respuesta no está en el contexto, responde: ‘No encuentro esta información en los documentos proporcionados.’"  
- "No inventes nombres, números ni referencias."  
  
El cuaderno `RAG.ipynb` extenderá esta idea con embeddings, bases de datos vectoriales y métricas de evaluación.

#### **8. Diseñar *prompts* que gestionen la incertidumbre**

Los modelos tienden a contestar siempre, incluso cuando no saben. Podemos:

- Incluir una opción "no lo sé" o "no está en el contexto".
- Pedir que marquen explícitamente cuando especulan.

Ejemplo de instrucción:

> "Si no estás seguro de la respuesta, di explícitamente:  
> ‘No lo sé con certeza; estoy especulando por estas razones: ...’"

Esto es especialmente importante en dominios sensibles (salud, finanzas, seguridad).



#### **9. Entender el patrón básico de agentes (pensar -> actuar -> observar -> responder)**

Los **agentes** combinan prompts + herramientas. El patrón conceptual es:

1. **Pensar:** el modelo razona internamente sobre la tarea.  
2. **Actuar:** decide usar una herramienta (búsqueda, API, código, etc.).  
3. **Observar:** lee la salida de la herramienta.  
4. **Responder:** genera la respuesta final al usuario.

LangChain formaliza este ciclo con *agents* y *tools*. Más adelante podemos conectar esto con el uso de RAG, bases de datos vectoriales o APIs externas.

#### **10. Diseñar *prompts* para evaluación automática (LLM-as-a-judge)**

Un LLM también puede evaluar respuestas de otros modelos, pero con cuidado:

- Definimos criterios claros (correctitud, claridad, cobertura, estilo).  
- Pedimos una salida estructurada (por ejemplo, puntuaciones 0-5 por criterio).  
- Recordamos que el juez también puede alucinar o sesgarse.

Ejemplo de formato de salida:

```json
{
  "score_correctitud": 0-5,
  "score_claridad": 0-5,
  "score_cobertura": 0-5,
  "comentarios": "texto libre"
}
```



#### **Ejemplos (pseudocódigos de técnicas de *prompting)***

Los siguientes fragmentos están pensados para **no demorar mucho** al ejecutarse:  
solo construyen *prompts* o muestran patrones. Puedes adaptarlos para usarlos con
el `pipeline` de Transformers o con LangChain.

**1. Prompt estructurado con secciones y delimitadores**

```python
def build_structured_prompt(tema: str) -> str:
    return f"""
[ROL]
Eres un profesor de IA que explica conceptos de forma clara y breve.

[TAREA]
Explica el tema: "{tema}" a un estudiante de pregrado.

[RESTRICCIONES]
- Máximo 2 párrafos.
- Incluye un ejemplo sencillo.

[FORMATO_SALIDA]
Escribe solo el texto de la explicación, sin listas ni viñetas.
"""

prompt = build_structured_prompt("transformers")
print(prompt)  # Aquí recién podrías pasarlo al modelo si quieres
```

Este patrón **no llama al modelo**: solo construye el prompt de forma legible y reutilizable.

**2. Few-shot/in-context learning con ejemplos breves**

```python
def build_fewshot_prompt(pregunta: str) -> str:
    ejemplos = """
Ejemplo 1
[IN]
Explica overfitting en una frase.
[OUT]
Overfitting es cuando un modelo se ajusta tanto a los datos de entrenamiento
que falla al generalizar a datos nuevos.

Ejemplo 2
[IN]
Explica regularización L2 en una frase.
[OUT]
La regularización L2 penaliza los pesos grandes para evitar que el modelo se
vuelva demasiado complejo.
"""

    return f"""
{ejemplos}

Ahora responde al siguiente caso:

[IN]
{pregunta}
[OUT]
"""

prompt = build_fewshot_prompt("Explica embeddings en una frase.")
print(prompt)
```

Puedes usar este prompt con tu `generate_text(prompt, max_new_tokens=60, do_sample=False)` para que siga el estilo de los ejemplos.

**3. *Chain-of-thought* (razonamiento paso a paso)**

```python
def build_cot_prompt(problema: str) -> str:
    return f"""
Resuelve el siguiente problema paso a paso.

Problema: {problema}

Instrucciones:
1. Primero, razona paso a paso explicando cada operación.
2. Luego, en la última línea, escribe:
   Respuesta final: <aquí solo el resultado>

Empieza tu razonamiento ahora.
"""

prompt = build_cot_prompt("Si un modelo acierta 80 de 100 ejemplos, ¿cuál es su accuracy?")
print(prompt)
```

Este patrón insiste en separar el **razonamiento** de la **respuesta final**.

**4. Auto-verificación y mejora iterativa**

```python
def build_self_check_prompt(texto: str) -> str:
    return f"""
Tienes el siguiente borrador de explicación:

---
{texto}
---

Tarea:
1. Identifica posibles errores, ambigüedades o partes poco claras.
2. Corrige el texto produciendo una versión mejorada.
3. Devuelve SOLO la versión mejorada, sin comentarios adicionales.
"""

draft = "Los transformers siempre usan atención completa y nunca se optimizan con SGD."
prompt = build_self_check_prompt(draft)
print(prompt)
```

La idea es que el modelo actúe como **revisor** de su propio texto u otro texto dado.

**5. Salida estructurada (JSON) para evaluación o uso posterior**

```python
def build_json_judge_prompt(respuesta: str, referencia: str) -> str:
    return f"""
Actúa como evaluador de respuestas cortas.

[REFERENCIA]
{referencia}
[/REFERENCIA]

[RESPUESTA_ESTUDIANTE]
{respuesta}
[/RESPUESTA_ESTUDIANTE]

Evalúa la respuesta del estudiante respecto a la referencia y
devuelve SOLO un JSON con el siguiente formato:

{{
  "score_correctitud": 0-5,
  "score_claridad": 0-5,
  "score_cobertura": 0-5,
  "comentarios": "texto breve"
}}
"""

prompt = build_json_judge_prompt(
    respuesta="Un transformer es una red neuronal que usa atención.",
    referencia="Un transformer es una arquitectura basada en atención que reemplazó a las RNN en muchas tareas de NLP."
)
print(prompt)
```

Este patrón se conecta con **LLM-as-a-judge** y fuerza una salida fácil de procesar con código.

**6. Prompting "grounded" con contexto al estilo RAG**

```python
def build_grounded_prompt(pregunta: str, contexto: str) -> str:
    return f"""
Responde a la pregunta usando EXCLUSIVAMENTE la información del contexto.

[CONTEXTO]
{contexto}
[/CONTEXTO]

Instrucciones:
- Si la información está en el contexto, responde de forma breve y precisa.
- Si NO está, responde exactamente:
  "No encuentro esta información en los documentos proporcionados."
- No inventes datos que no aparezcan en el contexto.

Pregunta: {pregunta}

Respuesta:
"""

contexto_demo = "RAG combina un módulo de recuperación de documentos con un modelo generativo para responder con contexto actualizado."
prompt = build_grounded_prompt("¿Qué es RAG?", contexto_demo)
print(prompt)
```

Este patrón es un mini-RAG "a mano": el contexto lo pasas tú, sin necesidad de una base vectorial todavía.


#### **Ejercicios**

**Ejercicio 1 - Historia interactiva con PromptTemplate + LLMChain**

**Basado en:** "Ejemplo 1: Generación de una historia interactiva".

1. **Analiza el prompt actual** que se pasa a `generate_text` en `continue_story(prompt, choice)`.

   * Escribe en una celda *markdown* qué partes corresponden a:

     * Contexto de la historia.
     * Instrucción al modelo.
     * Entrada del usuario (elección).
     * Restricciones (si es que hay).

2. **Diseña un prompt estructurado** usando el patrón de la sección de prompting:

   ```text
   [ROL]
   [CONTEXTO]
   [ELECCIÓN_DEL_USUARIO]
   [INSTRUCCIONES]
   [RESTRICCIONES]
   ```

   Escríbelo como una función:

   ```python
   def build_story_prompt(historia_actual: str, eleccion: str) -> str:
       ...
   ```

3. **Refactoriza `continue_story`** para usar `build_story_prompt` en lugar del prompt "plano" actual.

4. **Integra LangChain**:

   * Envuelve tu `generate_text` en un LLM de LangChain (por ejemplo `HuggingFacePipeline`).
   * Crea un `PromptTemplate` con variables `historia_actual` y `eleccion`.
   * Crea un `LLMChain` que reciba esas variables y devuelva el siguiente fragmento de historia.

5. **Comparación cualitativa**:

   * Ejecuta al menos 3 veces la historia con el código original y con la versión LangChain+PromptTemplate.
   * Escribe en markdown:

     * ¿Mejora la coherencia entre decisiones y resultado?
     * ¿Es más fácil ajustar el estilo (por ejemplo, hacerlo más humorístico, más oscuro, etc.)?

**Ejercicio 2 - Documento técnico con few-shot prompting**

**Basado en:** "Ejemplo 2: Generación de un documento técnico".

1. Observa cómo se generan actualmente las secciones (`Introducción`, `Aplicaciones`, etc.) con un prompt genérico.

2. Diseña un **prompt few-shot** para una sección (por ejemplo, "Aplicaciones"):

   * Escribe **2 ejemplos cortos** de textos bien escritos para otras secciones (por ejemplo, "Historia de la IA", "Limitaciones actuales").
   * Usa el formato:

     ```text
     [EJEMPLO 1]
     SECCIÓN: Historia de la IA
     TEXTO: ...

     [EJEMPLO 2]
     SECCIÓN: Limitaciones actuales
     TEXTO: ...

     [NUEVA_TAREA]
     SECCIÓN: {nombre_seccion}
     INSTRUCCIONES: Mantén el estilo de los ejemplos.
     ```

3. Implementa una función:

   ```python
   def build_tech_section_prompt(nombre_seccion: str, descripcion: str) -> str:
       ...
   ```

4. Usa LangChain para crear un `LLMChain` que, dado `nombre_seccion` y `descripcion`, genere el contenido de cada sección con estilo consistente.

5. **Discusión corta**:

   * ¿Qué diferencias notas entre:

     * Prompt sin ejemplos.
     * Prompt con pocos ejemplos (few-shot).

**Ejercicio 3 - Explicaciones matemáticas con *chain-of-thought***

**Basado en:** "Ejemplo 3: Explicaciones y ejemplos de funciones matemáticas".

1. El código actual pide explicaciones y ejemplos de conceptos matemáticos (funciones lineales, cuadráticas, etc.).

2. Diseña un **prompt con chain-of-thought**:

   ```text
   [ROL]
   Eres un profesor de matemáticas.

   [TAREA]
   Explica el concepto: {concepto}.

   [REQUISITOS]
   1. Primero, razona paso a paso de forma interna.
   2. Luego, entrega al estudiante:
      - Una explicación breve (2-3 párrafos).
      - Un ejemplo numérico resuelto.

   [FORMATO_SALIDA]
   Explicación:
   ...
   Ejemplo:
   ...
   ```

3. Implementa una función `build_math_prompt(concepto: str) -> str` y úsala con el `pipeline` actual.

4. Variante avanzada:

   * Escribe un prompt que le diga al modelo:

     > "Piensa paso a paso **pero no muestres** todo el razonamiento, solo entrega la explicación final y el ejemplo".
   * Compara las salidas con y sin esta instrucción:

     * ¿El modelo tiende a "pensar más" (más pasos, más detalles)?
     * ¿La explicación es más clara?

**Ejercicio 4 - Auto-verificación y mejora iterativa con LangChain**

**Idea:** aplicar la técnica de *self-checking* / *self-refinement*.

1. Elige una de las tareas anteriores (historia, sección técnica o explicación matemática).

2. Crea **dos prompts**:

   * `prompt_respuesta`: el modelo da una primera respuesta.
   * `prompt_revisor`: el modelo actúa como revisor de su propia respuesta, con instrucciones del tipo:

     > "Dado el enunciado y tu respuesta, detecta errores, mejora la claridad y corrige pasos dudosos. Devuelve una versión mejorada."

3. Implementa en LangChain un flujo de 2 pasos:

   * `LLMChain` 1: genera la respuesta inicial.
   * `LLMChain` 2: toma `{enunciado, respuesta_inicial}` y produce una respuesta mejorada.

4. **Actividad de análisis**:

   * Guarda (`print`) la respuesta inicial y la revisada para 3 entradas distintas.
   * Describe en markdown:

     * Casos donde la revisión mejora claramente la respuesta.
     * Casos donde la revisión introduce errores o cambios innecesarios.

**Ejercicio 5 - Prompting *grounded* al estilo mini-RAG**

**Basado en:** el pseudocódigo de `build_grounded_prompt` en el cuaderno.

1. Implementa por completo `build_grounded_prompt(pregunta: str, contexto: str)` siguiendo la idea del cuaderno:

   * Bloques `[ROL]`, `[CONTEXT]`, `[INSTRUCCIONES]`, `[RESPUESTA]`.
   * Regla explícita:

     > "Si la respuesta no está en el contexto, responde: 'No encuentro esta información en los documentos proporcionados.'"

2. Construye un pequeño "dataset" de contexto y preguntas, por ejemplo:

   ```python
   contexto_rag = """
   RAG combina un módulo de recuperación de documentos con un modelo
   generativo para responder usando información actualizada...
   """

   preguntas = [
       "¿Qué es RAG?",
       "¿Cómo se relaciona RAG con bases de datos vectoriales?",
       "¿Cuál es la capital de Francia?"
   ]
   ```

3. Para cada pregunta:

   * Construye el prompt con `build_grounded_prompt`.
   * Llama al `pipeline` y guarda la respuesta.

4. **Discute**:

   * ¿El modelo respeta la regla de "No encuentro esta información..." cuando preguntas algo fuera del contexto?
   * ¿Qué cambios harías en el prompt para reducir alucinaciones?

**Ejercicio 6 - Primer agentecon LangChain (pensar -> actuar -> observar -> responder)**

**Relacionado con:** sección "9. Patrón básico de agentes".

1. Define al menos **dos herramientas** como funciones Python simples:

   ```python
   def buscar_definicion(concepto: str) -> str:
       # Devuelve una "definición" desde un diccionario fijo en memoria
       ...

   def calcular_expresion(expr: str) -> str:
       # Evalúa una expresión matemática muy limitada, con seguridad
       ...
   ```

2. Con LangChain, crea:

   * Objetos `Tool` para cada función.
   * Un LLM (por ejemplo, el mismo HuggingFace o uno remoto tipo ChatOpenAI si lo tienes disponible).

3. Diseña un prompt de agente que siga el ciclo:

   ```text
   [ROL]
   Eres un agente que decide cuándo usar herramientas.

   [HERRAMIENTAS_DISPONIBLES]
   - buscar_definicion
   - calcular_expresion

   [INSTRUCCIONES]
   1. Piensa primero qué necesitas hacer.
   2. Si necesitas información externa, invoca una herramienta.
   3. Observa la salida.
   4. Solo entonces responde al usuario.
   ```

4. Prueba el agente con al menos 5 consultas:

   * Unas que requieran definiciones ("¿Qué es una función cuadrática?").
   * Otras que requieran cálculo ("Calcula 3x^2 - 4x - 5 para x=2").

5. Escribe en markdown:

   * ¿Cuándo decide usar una herramienta?
   * ¿Cuándo responde sin usar herramientas?
   * ¿Alguna decisión es claramente equivocada? ¿Cómo ajustarías el prompt del agente?

**Ejercicio 7 - LLM-as-a-judge: evaluación automática de respuestas**

**Relacionado con:** sección "10. Diseñar prompts para evaluación automática".

1. Define una tarea base, por ejemplo:

   * Explicar el concepto de "transformer".
   * Explicar qué es "RAG".

2. Pide al modelo una respuesta a la tarea con el pipeline o con LangChain.

3. Diseña un prompt de juez:

   ```text
   [ROL]
   Eres un evaluador estricto de respuestas de estudiantes.

   [CRITERIOS]
   - correctitud (0-5)
   - claridad (0-5)
   - cobertura (0-5)

   [TAREA]
   Evalúa la respuesta del estudiante a la siguiente pregunta:
   Pregunta: {pregunta}
   Respuesta_estudiante: {respuesta}

   [FORMATO_SALIDA]
   Devuelve un JSON válido con esta estructura:
   {{
     "score_correctitud": <número de 0 a 5>,
     "score_claridad": <número de 0 a 5>,
     "score_cobertura": <número de 0 a 5>,
     "comentarios": "texto breve"
   }}
   ```

4. Implementa una función:

   ```python
   def evaluar_respuesta(pregunta: str, respuesta: str) -> dict:
       ...
   ```

   que llame al modelo, intente parsear el JSON y devuelva un `dict` de Python.

5. Prueba con:

   * Una respuesta buena.
   * Una respuesta incompleta.
   * Una respuesta deliberadamente incorrecta.

6. Discusión:

   * ¿El juez es consistente?
   * ¿Qué riesgos ves al usar LLM-as-a-judge en un entorno de evaluación real?



In [ ]:
## Tus respuestas

**Mini-proyecto - Comparando "solo Transformers" vs "Transformers + LangChain + buenas técnicas de prompting"**

**Objetivo:** integrar todo.

1. Define un menú simple (puede ser con `input()` o en celdas separadas) con 3 "modos":

   * Modo A: historia interactiva.
   * Modo B: documento técnico corto.
   * Modo C: explicación matemática.

2. Para cada modo, implementa **dos versiones**:

   * Versión 1: usando solo el `pipeline` de `transformers` con prompts simples.
   * Versión 2: usando LangChain (PromptTemplate, LLMChain, quizá alguna `Tool`) y al menos **una técnica de prompting avanzada**:

     * few-shot,
     * chain-of-thought,
     * grounding con contexto,
     * auto-verificación, etc.

3. Pide a 2-3 compañeros (o tú mismo en distintos momentos) que:

   * Usen la versión 1 y la versión 2.
   * Evalúen subjetivamente: claridad, coherencia, utilidad.

4. Resume en markdown:

   * ¿Qué aporta LangChain en términos de organización del código y reusabilidad?
   * ¿Qué aportan las técnicas de prompting frente a prompts "ad-hoc"?
   * ¿En qué casos el *over-engineering* no se justifica (tarea demasiado simple)?


In [ ]:
## Tus respuestas